In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
%matplotlib inline

---
# Part 1

set parameters

In [2]:
NX    = 25
NY    = 25
XMIN  = 0
XMAX  = 1
YMIN  = 0
YMAX  = 1
NITER = 50  # max number of iterations

Initialize the grids

In [3]:
u = np.zeros(shape=(NX,NY))

set initial conditions (assume 0 everywhere)

In [4]:
def initial(u,nx,ny):
    u[:,:] = 0
    return u

Boundary conditions

In [5]:
def set_boundary_conditions(u,nx,ny):
    u[:, ny-1] = 1
    u[0, :]    = 0
    u[:, 0]    = 0
    u[nx-1, :] = 0
    

    #print("u is\n",u)
    """
    Input: u[i][j]
    
    Output: u[i][j]
    
    B.C.: top    (y=1): u = 1
          button (y=0): u = 0
          left   (x=0): u = 0
          right  (x=1): u = 0
    """

    # TODO: set the boundary conditions here
    
    return u

## Jacobi method
Do one iteration

In [6]:
def evolve_jacobi(u,nx,ny):
    
    """
    do one Jacobi iteration
    
    Inputs: u[size of NX][size of NY]
    NX: number of points in x-axis
    NY: number of points in y-axis
    
    Outputs: u
    
    """

    u = set_boundary_conditions(u, NX, NY)
    uold = u.copy()
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            u[i][j] = 0.25*(uold[i+1][j] + uold[i-1][j] 
                            + uold[i][j+1] + uold[i][j-1])

    return u

Check convergence by comparing the differecne between u1 and u2

In [7]:
def diff(u1,u2,nx,ny):
    error = 0.0
    for i in range(nx):
        for j in range(ny):
            error += abs(u1[i,j] - u2[i,j])
    return error

Start iterations

In [10]:
NX    = 10
NY    = 10
NITER = 50
u = np.zeros(shape=(NX,NY))
u = initial(u, NX, NY)
errors = np.zeros(NITER)
for n in range(NITER):
    
    plt.figure(1,figsize=(12,6), facecolor='white')
    plt.subplot(121)
    plt.title("N= {}".format(n))
    clear_output(wait=True)
    
    # make a copy of the u in the previous step
    uold = u.copy()
    
    # do one iteration
    u = evolve_jacobi(u, NX, NY)
    
    # check the difference
    err = diff(uold,u, NX, NY)
    errors[n] = err
    
    # plot the results
plt.imshow(u.T,origin='lower',extent=[XMIN,XMAX,YMIN,YMAX],interpolation='none')
plt.colorbar()
plt.subplot(122)
plt.plot(errors,'k-')
plt.xlim([0,NITER+1])
plt.xlabel("N")
plt.yscale('log')
#time.sleep(0.0001)
#plt.show()
plt.tight_layout()
plt.savefig('jacobi.png')
plt.close()
    

---
# Part 2

## Gauss-Seidel method

In [7]:
def evolve_gauss_seidel(u,nx,ny):
    u = set_boundary_conditions(u, nx, ny)
    uold = u.copy()
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            u[i][j] = 0.25*(u[i+1][j] + uold[i-1][j] 
                            + u[i][j+1] + uold[i][j-1])
    """
    do one Gauss-Seidel iteration
    """
    return u

In [25]:
#TODO: Visualize the results here
NX    = 100
NY    = 100
NITER = 100
u = np.zeros(shape=(NX,NY))
u = initial(u, NX, NY)
errors = np.zeros(NITER)

#u = niter_function(u, NX, NY, NITER, 'gs')

for n in range(NITER):
    
    # make a copy of the u in the previous step
    uold = u.copy()
    
    # do one iteration
    u = evolve_gauss_seidel(u, NX, NY)
    
    # check the difference
    err = 0.0
    err = diff(uold,u, NX, NY)
    errors[n] = err
    
# plot the results    
plt.figure(1,figsize=(12,6), facecolor='white')
plt.subplot(121)
plt.title("N= {}".format(n))
clear_output(wait=True)
plt.imshow(u.T,origin='lower',extent=[XMIN,XMAX,YMIN,YMAX],interpolation='none')
plt.colorbar()
plt.subplot(122)
plt.plot(errors,'k-')
plt.xlim([0,NITER+1])
plt.xlabel("N")
plt.yscale('log')
plt.tight_layout()
#time.sleep(0.0001)
plt.savefig('gs.png')
plt.close()

---
# Part 3

## Successive over-relax method:

w < 1 : under-relaxation
w = 1 : Gauss-Seidel method
w > 1 : over-relaxation

In [9]:
def sor(w,u,nx,ny):
    u = set_boundary_conditions(u, nx, ny)
    uold = u.copy()
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            ug = evolve_gauss_seidel(u, nx, ny)
            u[i][j] = (1-w)*uold[i][j] + w*ug[i][j]
    """
    do one Gauss-Seidel iteration
    """
    return u

In [20]:
#TODO: Visualize the results here
NX    = 100
NY    = 100
NITER = 100
u = np.zeros(shape=(NX,NY))
u = initial(u, NX, NY)
w = 1.2
errors = np.zeros(NITER)
for n in range(NITER):

    # make a copy of the u in the previous step
    uold = u.copy()
    
    # do one iteration
    u = sor(w, u, NX, NY)
    
    # check the difference
    err = diff(uold,u, NX, NY)
    errors[n] = err
    
# plot the results
plt.figure(1,figsize=(12,6), facecolor='white')
plt.subplot(121)
plt.title("N= {}".format(NITER))
clear_output(wait=True)
plt.imshow(u.T,origin='lower',extent=[XMIN,XMAX,YMIN,YMAX],interpolation='none')
plt.colorbar()
plt.subplot(122)
plt.plot(errors,'k-')
plt.xlim([0,NITER+1])
plt.xlabel("N")
plt.yscale('log')
plt.tight_layout()
plt.savefig('sor.png')
plt.close()
    

KeyboardInterrupt: 

---
# Part 4

## Multi-grid method

In [9]:
NX1   = 25
NY1   = 25
NX2   = 50
NY2   = 50
NX3   = 100
NY3   = 100
NITER = 100
u1 = np.zeros(shape=(NX1,NY1))
u2 = np.zeros(shape=(NX2,NY2))
u3 = np.zeros(shape=(NX3,NY3))
XMIN  = 0
XMAX  = 1
YMIN  = 0
YMAX  = 1

In [46]:
def restriction(u_fine,u_coarser,NXC,NYC):
    """
    from fine grid to coarser grid.
    Assume the gird size is different by a factor of 2
    
    update the coarser grid based on information of the fine grid
    
    Inputs:
    
    u_fine: size of 2 NXC and 2 NYC
    u_coarser: size of NXC and NYC
    
    
    Outputs:
    
    u: size of NXC and NYC
    
    
    """
    a = np.zeros(shape=(2*NXC,2*NYC)) # inputted fine matrix
    a = u_fine.copy()
    b = np.zeros(shape=(2*NXC,NYC))
    A = np.zeros(shape=(NXC,NYC)) # output, coarser one
    uout = np.zeros(shape=(NXC,NYC))
    
    for i in range(2*NXC):
        for j in range(2*NYC):
            if j % 2 == 0:
                k = int(j/2)
                #print(k)
                b[i][k] = a[i][j] + a[i][j+1]
    
    for j in range(NYC):
        for i in range(2*NXC):
            if i % 2 ==0:
                k = int(i/2)
                A[k][j] += b[i][j] + b[i+1][j]
                A[k][j] = 0.25*A[k][j]
                
    uout = A.copy()
    #print(f"uout is {uout}")
    
    return uout  

In [47]:
def prolongation(u_fine, u_coarser,NXC, NYC):
    """
    from coarser grid to fine grid
    
    update the fine grid based on information of the coarser grid
    
    Inputs:
    
    u_fine: size of 2 NXC and 2 NYC
    u_coarser: size of NXC and NYC
    
    
    Outputs:
    
    u: size of 2 NXC and 2 NYC
    
    """
    a = np.zeros(shape=(NXC,NYC)) # inputted coarser matrix
    a = u_coarser.copy()
    A = np.zeros(shape=(2*NXC,2*NYC)) # output, fine one
    uout = np.zeros(shape=(2*NXC,2*NYC))
    
    for i in range(NXC):
        for j in range(NYC):
            i2 = 2*i
            j2 = 2*j
            A[i2][j2]     = a[i][j]
            A[i2][j2+1]   = a[i][j]
            A[i2+1][j2]   = a[i][j]
            A[i2+1][j2+1] = a[i][j] 
    uout = A.copy()
    
    return uout

In [48]:
#TODO: To the full multigird cycle here and Visualize the results

In [49]:
def niter_function(u, nx, ny, n, step):
    errors = np.zeros(n)
    #uold = u.copy()
    for i in range(n):
        uold = u.copy()
        err = 0
        u = evolve_gauss_seidel(u, nx, ny)
        err = diff(uold, u, nx, ny)
        errors[i] = err
    
    plt.figure(1, figsize=(12,6), facecolor='white')
    plt.subplot(121)
    clear_output(wait=True)
    plt.title("N= {}".format(step))
    plt.imshow(u.T,origin='lower',extent=[XMIN,XMAX,YMIN,YMAX],interpolation='none')
    plt.colorbar()
    
    plt.subplot(122)
    plt.plot(errors,'k-')
    plt.xlim([0,n+1])
    plt.xlabel("N")
    plt.yscale('log')
    plt.tight_layout()
    plt.savefig('n={}'.format(step))
    plt.close()
    
    uout = u.copy()
    return uout

In [55]:
NITER = 100
u1 = initial(u1, NX1, NY1)
u2 = initial(u2, NX2, NY2)
u3 = initial(u3, NX3, NY3)
# full multigrid: u1, u2, u1, u2, u3, u2, u1, u2, u3

#print(f"u1 = {u1}")
u1 = set_boundary_conditions(u1, NX1, NY1)
'''
plt.figure(1,figsize=(12,5))
clear_output(wait=True)
plt.imshow(u1.T,origin='lower',extent=[XMIN,XMAX,YMIN,YMAX],interpolation='none')
plt.colorbar()
plt.savefig('u1 boundary.png')
plt.close()
'''
u1 = niter_function(u1, NX1, NY1, NITER, 1)
#print(f"u1 = {u1}")

u2 = set_boundary_conditions(u2, NX2, NY2)
'''
plt.figure(1,figsize=(12,5))
clear_output(wait=True)
plt.imshow(u2.T,origin='lower',extent=[XMIN,XMAX,YMIN,YMAX],interpolation='none')
plt.colorbar()
plt.savefig('u2 boundary.png')
plt.close()
'''

u2 = prolongation(u2, u1, NX1, NY1)
'''
plt.figure(1,figsize=(12,5))
clear_output(wait=True)
plt.imshow(u2.T,origin='lower',extent=[XMIN,XMAX,YMIN,YMAX],interpolation='none')
plt.colorbar()
plt.savefig('check.png')
'''
#print(f"u2= {u2}")
u2 = niter_function(u2, NX2, NY2, NITER, 2)

u1 = restriction(u2, u1, NX1, NY1)
u1 = niter_function(u1, NX1, NY1, NITER, 3)

u2 = prolongation(u2, u1, NX1, NY1)
u2 = niter_function(u2, NX2, NY2, NITER, 4)


u3 = set_boundary_conditions(u3, NX3, NY3)
'''
plt.figure(1,figsize=(12,5))
clear_output(wait=True)
plt.imshow(u3.T,origin='lower',extent=[XMIN,XMAX,YMIN,YMAX],interpolation='none')
plt.colorbar()
plt.savefig('u3 boundary.png')
plt.close()
'''

u3 = prolongation(u3, u2, NX2, NY2)
u3 = niter_function(u3, NX3, NY3, NITER, 5)

u2 = restriction(u3, u2, NX2, NY2)
u2 = niter_function(u2, NX2, NY2, NITER, 6)

u1 = restriction(u2, u1, NX1, NY1)
u1 = niter_function(u1, NX1, NY1, NITER, 7)

u2 = prolongation(u2, u1, NX1, NY1)
u2 = niter_function(u2, NX2, NY2, NITER, 8)

u3 = prolongation(u3, u2, NX2, NY2)
u3 = niter_function(u3, NX3, NY3, NITER, 9)

print('Done!')

Done!
